<b>
Name : Anish Rajesh Adnani
<br>
USC ID : 4092610491
<br>
USC Email ID : adnani@usc.edu
<br>
Github Username : midnightbot
<br>
    </b>

<b> (1) Time Series Classification Part 1: Feature Creation/Extraction <b>


<b> An interesting task in machine learning is classification of time series. In this problem,
we will classify the activities of humans based on time series obtained by a Wireless
Sensor Network. </b>

In [97]:
## Basic imports
import pandas as pd  ## reading the data
import openpyxl  ## Optional dependency for reading excel using pandas
import bootstrapped.bootstrap as bs  ## used for calculating confidence intervals
import numpy as np  ## used for numpy arrays
import bootstrapped.stats_functions as bs_stats  ## used for calculating std
import copy  ## used to make a copy of list
%matplotlib inline

<b> 1 .(a) Download the AReM data from:
https://archive.ics.uci.edu/ml/datasets/
Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\
%29
. The dataset contains 7 folders that represent seven types of activities. In
each folder, there are multiple files each of which represents an instant of a human
performing an activity.
1
Each file containis 6 time series collected from activities
of the same person, which are called avg
rss12, var
rss12, avg
rss13, var
rss13,
vg
rss23, and ar
rss23. There are 88 instances in the dataset, each of which con-
tains 6 time series and each time series has 480 consecutive values </b>

In [2]:
## Data Consists of 7 folders -> 7 types of activities
## each folder has multiple files, each representing an instant of human performing an activity
## each file has 6 timeseries of the same person
## total there are 88 instances in dataset (88 files in the dataset)
## each file has 480 timeseries values

In [3]:
data_folder_location = '../data/AReM/'
data_folders = ["bending1", "bending2", "cycling", "lying", "sitting", "standing", "walking"]
test_data_ranges = [2, 2, 3, 3, 3, 3, 3]
train_data_ranges = [7, 6, 15, 15, 15, 15, 15]

<b> 1.(b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1,
2, and 3 in other folders as test data and other datasets as train data </b>

In [48]:
def custom_csv_file_reader(fileName):
    """
    Reads the csv file (space seperated or comma seperated) and returns the DataFrame
    Parameters : fileName (A Path Containing some csv Dataset)
    Return : file_data (DataFrame)
    """
    df_a = pd.read_csv(fileName, skiprows=482, sep=','  , engine='python')
    df_b = pd.read_csv(fileName, skiprows=482, sep='\s+'  , engine='python')
    #print(fileName)
    #print(df_a)
    ## csv file is comma seperated
    if df_a.shape[1] == 7:
        file_data = pd.read_csv(fileName, skiprows = 4, sep=','  , engine='python')
        
        
    ## csv file is space seperated
    if df_b.shape[1] == 7:
        file_data = pd.read_csv(fileName, skiprows = 5, sep='\s+'  , engine='python', header=None)
        file_data.columns = ["time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"]
    #print(file_data.columns)
    return file_data
        
        

In [49]:
print("Started Creating Train and Test Data")
print("=====================================================================================\n")
train_data = {}
test_data = {}
full_data = {}
"""
skip the first 4 rows which contains the following
Task
Frequency (Hz)
Clock (millisecond)
Duration (seconds)
NOTE ** -> cycling folder dataset9.csv, cycling folder dataset14.csv had extra commas in the last line
"""

## making the test_data
for x in range(len(data_folders)):
    for y in range(1, test_data_ranges[x] + 1):
        file_name = data_folder_location + data_folders[x] + "/" + "dataset" + str(y) + ".csv"
        file_data = custom_csv_file_reader(file_name)
        test_data[file_name] = file_data
        
        
print("All Test Data Files are")
print(list(test_data.keys()))
print("=====================================================================================\n")
## making the train_data
for x in range(len(data_folders)):
    for y in range(test_data_ranges[x]+1, train_data_ranges[x]+1):
        file_name = data_folder_location + data_folders[x] + "/" + "dataset" + str(y) + ".csv"
        file_data = custom_csv_file_reader(file_name)
        train_data[file_name] = file_data
        

## making the full_data
for x in range(len(data_folders)):
    for y in range(1, train_data_ranges[x]+1):
        file_name = data_folder_location + data_folders[x] + "/" + "dataset" + str(y) + ".csv"
        file_data = custom_csv_file_reader(file_name)
        full_data[file_name] = file_data
        
print("All Train Data Files are")
print(list(train_data.keys()))
print("=====================================================================================\n")

Started Creating Train and Test Data

All Test Data Files are
['../data/AReM/bending1/dataset1.csv', '../data/AReM/bending1/dataset2.csv', '../data/AReM/bending2/dataset1.csv', '../data/AReM/bending2/dataset2.csv', '../data/AReM/cycling/dataset1.csv', '../data/AReM/cycling/dataset2.csv', '../data/AReM/cycling/dataset3.csv', '../data/AReM/lying/dataset1.csv', '../data/AReM/lying/dataset2.csv', '../data/AReM/lying/dataset3.csv', '../data/AReM/sitting/dataset1.csv', '../data/AReM/sitting/dataset2.csv', '../data/AReM/sitting/dataset3.csv', '../data/AReM/standing/dataset1.csv', '../data/AReM/standing/dataset2.csv', '../data/AReM/standing/dataset3.csv', '../data/AReM/walking/dataset1.csv', '../data/AReM/walking/dataset2.csv', '../data/AReM/walking/dataset3.csv']

All Train Data Files are
['../data/AReM/bending1/dataset3.csv', '../data/AReM/bending1/dataset4.csv', '../data/AReM/bending1/dataset5.csv', '../data/AReM/bending1/dataset6.csv', '../data/AReM/bending1/dataset7.csv', '../data/AReM/be

<b> 1. (c) Feature Extraction
Classification of time series usually needs extracting features from them. In this
problem, we focus on time-domain features.</b>

<b> 1. (c) (i) Research what types of time-domain features are usually used in time series
classification and list them (examples are minimum, maximum, mean, etc).</b>

<b> Reference https://stats.stackexchange.com/questions/50807/features-for-time-series-classification

** Simple Statistical Features
- Mean
- Median
- Standard Deviation
- Minimum Value
- Maximum Value
- Skewness, Kurtosis
    
** Time Series Analysis Related Features
- Cross-Correlations
- Auto-Correlations
- Parameters of AR part
- Parameters of MA part
</b>

<b> 1.(c) (ii) Extract the time-domain features minimum, maximum, mean, median, stan-
dard deviation, first quartile, and third quartile for all of the 6 time series
in each instance. You are free to normalize/standardize features or use them
directly</b>

In [59]:
## new data will have 88 lines (1 line for each file)
## new data will have 42 columns 
## (min,max, mean, median,std,1st quartile, 3rd quartile) * for features (avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23)
dataset_columns = []
time_domain_fts = ["min", "max", "mean","median" ,"std", "first_quartile", "third_quartile"]
columnnames = test_data["../data/AReM/bending1/dataset1.csv"].columns
columnnames = list(columnnames[1:])

print(columnnames)
print("======================================\n")

for x in columnnames:
    for y in time_domain_fts:
        dataset_columns.append(y+"_"+x)
print(dataset_columns)      
print(len(dataset_columns))
print("======================================\n")



all_rows = []
for dfs in full_data:
    this_row = []
    for cols in columnnames:
        arr = full_data[dfs][cols]
    
        min_of_arr = arr.min()
        max_of_arr = arr.max()
        mean_of_arr = arr.mean()
        median_of_arr = arr.median()
        std_of_arr = arr.std()
        first_quartile_of_arr = arr.quantile(0.25)
        third_quartile_of_arr = arr.quantile(0.75)
        
        this_ft_data = [min_of_arr, max_of_arr, mean_of_arr,median_of_arr ,std_of_arr, first_quartile_of_arr, third_quartile_of_arr]
        
        this_row+=this_ft_data
    all_rows.append(this_row)        
    
print("Time Domain Feature Data ==>")
time_domain_feature_data = pd.DataFrame(data=all_rows, columns = dataset_columns)
print(time_domain_feature_data)
print("Time Domain Feature Data Shape ==> ", time_domain_feature_data.shape)


['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

['min_avg_rss12', 'max_avg_rss12', 'mean_avg_rss12', 'median_avg_rss12', 'std_avg_rss12', 'first_quartile_avg_rss12', 'third_quartile_avg_rss12', 'min_var_rss12', 'max_var_rss12', 'mean_var_rss12', 'median_var_rss12', 'std_var_rss12', 'first_quartile_var_rss12', 'third_quartile_var_rss12', 'min_avg_rss13', 'max_avg_rss13', 'mean_avg_rss13', 'median_avg_rss13', 'std_avg_rss13', 'first_quartile_avg_rss13', 'third_quartile_avg_rss13', 'min_var_rss13', 'max_var_rss13', 'mean_var_rss13', 'median_var_rss13', 'std_var_rss13', 'first_quartile_var_rss13', 'third_quartile_var_rss13', 'min_avg_rss23', 'max_avg_rss23', 'mean_avg_rss23', 'median_avg_rss23', 'std_avg_rss23', 'first_quartile_avg_rss23', 'third_quartile_avg_rss23', 'min_var_rss23', 'max_var_rss23', 'mean_var_rss23', 'median_var_rss23', 'std_var_rss23', 'first_quartile_var_rss23', 'third_quartile_var_rss23']
42

Time Domain Feature Data ==>
    min_avg_rss1

<b> 1. (c) (iii) Estimate the standard deviation of each of the time-domain features you
extracted from the data. Then, use Python’s bootstrapped or any other
method to build a 90% bootsrap confidence interval for the standard deviation
of each feature</b>

In [104]:
temp_arr = []
for i in time_domain_feature_data.columns:
    col_data = time_domain_feature_data[i]
    std_of_col = col_data.std()
    temp_arr.append([i,std_of_col])
    
print("Standard Deviation of all Time Domain Features == > ")
display_pd = pd.DataFrame(columns = ["Time Domain Feature", "Standard Deviation"], data = temp_arr)
display_pd

Standard Deviation of all Time Domain Features == > 


,Time Domain Feature,Standard Deviation
0,min_avg_rss12,9.569975
1,max_avg_rss12,4.394362
2,mean_avg_rss12,5.335718
3,median_avg_rss12,5.440054
4,std_avg_rss12,1.772153
5,first_quartile_avg_rss12,6.153590
6,third_quartile_avg_rss12,5.138925
7,min_var_rss12,0.000000
8,max_var_rss12,5.062729
9,mean_var_rss12,1.574164


In [105]:
temp_arr2 = []
lower_bound = []
upper_bound = []
for i in time_domain_feature_data.columns:
    col_data = time_domain_feature_data[i]
    col_data = np.array(col_data)
    bootstrap_confidence_interval = bs.bootstrap(col_data, stat_func = bs_stats.std, alpha = 0.1)
    lower_bound.append(bootstrap_confidence_interval.lower_bound)
    upper_bound.append(bootstrap_confidence_interval.upper_bound)
    
temp_arr2 = copy.deepcopy(temp_arr)
for x in range(len(temp_arr2)):
    temp_arr2[x].append(lower_bound[x])
    temp_arr2[x].append(upper_bound[x])
    
print("90% Confidence Interval of all Time Domain Features ==> ")
display_pd1 = pd.DataFrame(columns = ["Time Domain Feature", "Standard Deviation", "Lower Bound", "Upper Bound"], data = temp_arr2)
display_pd1

90% Confidence Interval of all Time Domain Features ==> 


,Time Domain Feature,Standard Deviation,Lower Bound,Upper Bound
0,min_avg_rss12,9.569975,8.326318,10.821333
1,max_avg_rss12,4.394362,3.488314,5.426319
2,mean_avg_rss12,5.335718,4.755663,5.919317
3,median_avg_rss12,5.440054,4.846076,6.051306
4,std_avg_rss12,1.772153,1.585878,1.960743
5,first_quartile_avg_rss12,6.153590,5.636140,6.694815
6,third_quartile_avg_rss12,5.138925,4.400884,5.897185
7,min_var_rss12,0.000000,0.000000,0.000000
8,max_var_rss12,5.062729,4.689237,5.463716
9,mean_var_rss12,1.574164,1.435520,1.745184


<b> 1. (c) (iv) Use your judgement to select the three most important time-domain features
(one option may be min, mean, and max)</b>

Mean : Whenever the data is continous, the mean tends to provide a general overall idea of the data

Standard Deviation : Standard Deviation will help us measure the dispersion of data from the mean, basically roughly helps us estimate how the data is dispersed

Median : Gives the central point of the data which will help us get a central idea about the data. It is also less affected by outliers/noise

<b> Q.2 ISLR 3.7.4</b>

https://github.com/DSCI-552/homework-3-midnightbot/blob/main/Adnani_Anish_HW3_ISLR3.7.4.pdf